In [ ]:
import yfinance as yf
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output

# Ações para análise
acoes = ['AAPL', 'MSFT']

# Obtendo os dados históricos das ações
dados_acoes = yf.download(acoes, start='2022-01-01', end='2023-01-01')

# Criando o aplicativo Flask com Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout do aplicativo
app.layout = html.Div([
    html.H1("Gráfico de Candles das Ações AAPL e MSFT"),
    dcc.Graph(id='candlestick-plot')
])

@app.callback(
    Output('candlestick-plot', 'figure'),
    [Input('candlestick-plot', 'relayoutData')]
)
def update_candlestick_plot(relayout_data):
    return {
        'data': [
            go.Candlestick(
                x=dados_acoes.index,
                open=dados_acoes[acao]['Open'],
                high=dados_acoes[acao]['High'],
                low=dados_acoes[acao]['Low'],
                close=dados_acoes[acao]['Adj Close'],
                name=acao
            )
            for acao in acoes
        ],
        'layout': {
            'title': 'Gráfico de Candles das Ações AAPL e MSFT',
            'xaxis': {'title': 'Data'},
            'yaxis': {'title': 'Preço de Fechamento'}
        }
    }

if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
import yfinance as yf
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html

# Obtendo os dados históricos da ação do Facebook
acao_facebook = yf.download('FB', start='2022-01-01', end='2023-01-01')

# Criando o aplicativo Flask com Dash
app = dash.Dash(__name__)

# Layout do aplicativo
app.layout = html.Div([
    html.H1("Gráfico de Candles da Ação do Facebook (FB)"),
    dcc.Graph(
        id='candlestick-plot',
        figure={
            'data': [
                go.Candlestick(
                    x=acao_facebook.index,
                    open=acao_facebook['Open'],
                    high=acao_facebook['High'],
                    low=acao_facebook['Low'],
                    close=acao_facebook['Close'],
                    name='FB'
                )
            ],
            'layout': {
                'title': 'Gráfico de Candles da Ação do Facebook (FB)',
                'xaxis': {'title': 'Data'},
                'yaxis': {'title': 'Preço'},
                'yaxis2': {'title': 'Volume', 'overlaying': 'y', 'side': 'right'}
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from flask import Flask, render_template

# Obtendo os dados históricos da ação do Facebook
acao_facebook = yf.download('FB', start='2022-01-01', end='2023-01-01')

# Criando o aplicativo Flask
app = Flask(__name__)

# Rota para renderizar o gráfico de candles da ação do Facebook
@app.route('/')
def plot_candlestick():
    fig = go.Figure(data=[go.Candlestick(x=acao_facebook.index,
                                        open=acao_facebook['Open'],
                                        high=acao_facebook['High'],
                                        low=acao_facebook['Low'],
                                        close=acao_facebook['Close'])])

    fig.update_layout(title='Gráfico de Candles da Ação do Facebook (FB)',
                      xaxis_title='Data',
                      yaxis_title='Preço',
                      yaxis2_title='Volume',
                      yaxis2=dict(overlaying='y', side='right'))

    return render_template('candlestick_plot.html', plot=fig.to_html())

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import yfinance as yf
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from flask import Flask, render_template

# Obtendo os dados históricos da ação do Facebook
acao_facebook = yf.download('FB', start='2022-01-01', end='2023-01-01')

# Criando o aplicativo Flask
app = Flask(__name__)

# Rota para renderizar o gráfico de candles da ação do Facebook
@app.route('/')
def plot_candlestick():
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

    # Gráfico de candles
    fig.add_trace(go.Candlestick(x=acao_facebook.index,
                                 open=acao_facebook['Open'],
                                 high=acao_facebook['High'],
                                 low=acao_facebook['Low'],
                                 close=acao_facebook['Close'],
                                 name='FB'), row=1, col=1)

    # Gráfico de volume
    fig.add_trace(go.Bar(x=acao_facebook.index,
                         y=acao_facebook['Volume'],
                         name='Volume'), row=2, col=1)

    fig.update_layout(title='Gráfico de Candles e Volume da Ação do Facebook (FB)',
                      xaxis_title='Data',
                      yaxis_title='Preço',
                      yaxis2_title='Volume',
                      showlegend=False)

    return render_template('candlestick_plot.html', plot=fig.to_html())

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
import yfinance as yf
from bokeh.io import curdoc
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.models.widgets import Select
from datetime import datetime

# Lista de ações para análise
acoes = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']

# Obtendo os dados históricos das ações
dados_acoes = yf.download(acoes, start='2022-01-01', end='2023-01-01')

# Criando a fonte de dados para o gráfico
source = ColumnDataSource(dados_acoes['Close'])

# Criando o gráfico de linha para cada ação
p = figure(x_axis_type='datetime', title='Preço de Fechamento das Ações', width=800, height=400)
for acao in acoes:
    p.line(x='Date', y=acao, source=source, line_width=2, legend_label=acao)

# Adicionando as ferramentas de interação
hover = HoverTool()
hover.tooltips = [("Data", "@Date{%F}"), ("Preço de Fechamento", "$y")]
hover.formatters = {'@Date': 'datetime'}
p.add_tools(hover)

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'

# Criando o menu dropdown para selecionar a ação
menu = Select(options=acoes, value='AAPL', title='Selecione uma Ação')

# Função para atualizar o gráfico ao selecionar uma ação no menu dropdown
def update_plot(attr, old, new):
    selected_acao = menu.value
    p.title.text = f"Preço de Fechamento da Ação {selected_acao}"
    source.data = ColumnDataSource(dados_acoes[selected_acao]).data

menu.on_change('value', update_plot)

# Organizando o layout do dashboard
layout = column(menu, p)

# Configurando o documento do Bokeh
curdoc().add_root(layout)


In [ ]:
import yfinance as yf
import plotly.graph_objects as go

# Lista de ações para análise
acoes = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']

# Obtendo os dados históricos das ações
dados_acoes = yf.download(acoes, start='2023-01-01', end='2023-06-01')

# Função para criar o gráfico de candles
def criar_grafico_candles(acao):
    candlestick = go.Candlestick(x=dados_acoes.index,
                                 open=dados_acoes[acao]['Open'][0],
                                 high=dados_acoes[acao]['High'][1],
                                 low=dados_acoes[acao]['Low'][2],
                                 close=dados_acoes[acao]['Close'][3],
                                 name=acao)

    return candlestick

# Criando lista com os gráficos de candles para cada ação
graficos_candles = [criar_grafico_candles(acao) for acao in acoes]

# Layout do gráfico
layout = go.Layout(title='Gráficos de Candles das Ações',
                   xaxis=dict(title='Data'),
                   yaxis=dict(title='Preço'))

# Criando a figura com os gráficos de candles
figura = go.Figure(data=graficos_candles, layout=layout)

# Exibindo o gráfico
figura.show()


In [ ]:
import yfinance as yf
import plotly.graph_objects as go

# Lista de ações para análise
acoes = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']

# Obtendo os dados históricos das ações
dados_acoes = yf.download(acoes, start='2022-01-01', end='2023-01-01')

dados_acoes

In [ ]:
import yfinance as yf
import plotly.graph_objects as go

# Ação para análise
acao = 'AAPL'  # Altere para a ação desejada

# Obtendo os dados históricos da ação
dados_acao = yf.download(acao, start='2022-01-01', end='2023-01-01')

# Criando o gráfico de candles
candlestick = go.Candlestick(x=dados_acao.index,
                             open=dados_acao['Open'],
                             high=dados_acao['High'],
                             low=dados_acao['Low'],
                             close=dados_acao['Close'],
                             name=acao)

# Layout do gráfico
layout = go.Layout(title=f'Gráfico de Candles da Ação {acao}',
                   xaxis=dict(title='Data'),
                   yaxis=dict(title='Preço'))

# Criando a figura com o gráfico de candles
figura = go.Figure(data=[candlestick], layout=layout)

# Exibindo o gráfico
figura.show()


In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from flask import Flask, render_template

# Ação para análise
acao = 'AAPL'  # Altere para a ação desejada

# Obtendo os dados históricos da ação
dados_acao = yf.download(acao, start='2022-01-01', end='2023-01-01')

# Criando o gráfico de candles
candlestick = go.Candlestick(x=dados_acao.index,
                             open=dados_acao['Open'],
                             high=dados_acao['High'],
                             low=dados_acao['Low'],
                             close=dados_acao['Close'],
                             name=acao)

# Layout do gráfico
layout = go.Layout(title=f'Gráfico de Candles da Ação {acao}',
                   xaxis=dict(title='Data'),
                   yaxis=dict(title='Preço'))

# Criando a figura com o gráfico de candles
figura = go.Figure(data=[candlestick], layout=layout)

# Criando o aplicativo Flask
app = Flask(__name__)

# Rota para renderizar o template com o gráfico
@app.route('/')
def renderizar_template():
    # Converte a figura para HTML e passa para o template
    grafico_html = figura.to_html(full_html=False, include_plotlyjs='cdn')
    return render_template('grafico.html', grafico_html=grafico_html)

if __name__ == '__main__':
    app.run(debug=True)


In [3]:
import yfinance as yf

dados_acao = yf.download('APPL', start='2023-01-01', end='2023-06-01')
dados_acao

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- APPL: No data found for this date range, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [2]:
from datetime import datetime

In [3]:
datetime.today()

datetime.datetime(2023, 8, 2, 21, 57, 22, 692888)

In [4]:
import yfinance as yf

start = '2010-06-01'
end = datetime.today()

dados = yf.download('META', start=start, end=end)
dados

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-05-18,42.049999,45.000000,38.000000,38.230000,38.230000,573576400
2012-05-21,36.529999,36.660000,33.000000,34.029999,34.029999,168192700
2012-05-22,32.610001,33.590000,30.940001,31.000000,31.000000,101786600
2012-05-23,31.370001,32.500000,31.360001,32.000000,32.000000,73600000
2012-05-24,32.950001,33.209999,31.770000,33.029999,33.029999,50237200
...,...,...,...,...,...,...
2023-07-27,325.119995,325.350006,309.839996,311.709991,311.709991,64229200
2023-07-28,316.880005,326.200012,314.250000,325.480011,325.480011,39220300
2023-07-31,323.690002,325.660004,317.589996,318.600006,318.600006,25799600


In [1]:
from datetime.datetime import today

today()

ModuleNotFoundError: No module named 'datetime.datetime'; 'datetime' is not a package